In [42]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa
from tqdm import tqdm
import random

# Augmentadores: espejo horizontal + zoom aleatorio (1.0 a 1.2)
flip_augmenter_horizontal = iaa.Fliplr(1.0)  # 100% de probabilidad de aplicar espejo
flip_augmenter_vertical = iaa.Flipud(1.0) 

# Directorios base
base_dirs = ["train", "valid", "test"]
output_dir = "augmented_data"

# Procesar cada conjunto de datos (train, validation, test)
for base_dir in base_dirs:
    base_path = os.path.join(".", base_dir)
    if not os.path.exists(base_path):
        continue  # Saltar si el directorio no existe

    for class_name in os.listdir(base_path):
        class_path = os.path.join(base_path, class_name)
        images_path = os.path.join(class_path, "Images")
        labels_path = os.path.join(class_path, "labels")

        if not os.path.exists(images_path) or not os.path.exists(labels_path):
            continue  # Saltar si no existen ambas carpetas

        # Crear carpetas de salida
        output_class_images = os.path.join(output_dir, base_dir, class_name, "Images")
        output_class_labels = os.path.join(output_dir, base_dir, class_name, "labels")
        os.makedirs(output_class_images, exist_ok=True)
        os.makedirs(output_class_labels, exist_ok=True)

        # Procesar cada imagen
        for img_name in tqdm(os.listdir(images_path), desc=f"Procesando {class_name} en {base_dir}"):
            if not img_name.endswith((".jpg", ".png")):
                continue  # Saltar archivos no imagen
            
            img_path = os.path.join(images_path, img_name)
            label_path = os.path.join(labels_path, img_name.replace(".jpg", ".txt").replace(".png", ".txt"))

            img = cv2.imread(img_path)
            if img is None or not os.path.exists(label_path):
                continue  # Saltar imágenes corruptas o sin etiquetas

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            height, width, _ = img.shape

            # Leer etiquetas YOLO
            with open(label_path, "r") as f:
                lines = f.readlines()

            boxes = []
            class_labels = []
            for line in lines:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center, y_center, w, h = map(float, parts[1:])
                boxes.append([x_center, y_center, w, h])
                class_labels.append(class_id)

            # Aplicar espejo horizontal (solo espejo)
            flipped_img = flip_augmenter_horizontal(image=img)

            # Ajustar bounding boxes para el espejo
            flipped_boxes = [[1.0 - x, y, w, h] for x, y, w, h in boxes]

            # Guardar imagen con solo el espejo
            output_img_name_flip = f"flipH_{img_name}"
            output_img_path_flip = os.path.join(output_class_images, output_img_name_flip)
            cv2.imwrite(output_img_path_flip, cv2.cvtColor(flipped_img, cv2.COLOR_RGB2BGR))

            # Guardar etiquetas YOLO ajustadas para el espejo
            output_label_name_flip = output_img_name_flip.replace(".jpg", ".txt").replace(".png", ".txt")
            output_label_path_flip = os.path.join(output_class_labels, output_label_name_flip)
            with open(output_label_path_flip, "w") as f:
                for class_id, (x, y, w, h) in zip(class_labels, flipped_boxes):
                    f.write(f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

            img_both = cv2.imread(output_img_path_flip)
            if img_both is None or not os.path.exists(output_label_path_flip):
                continue  # Saltar imágenes corruptas o sin etiquetas

            img_both = cv2.cvtColor(img_both, cv2.COLOR_BGR2RGB)
            height, width, _ = img_both.shape

            # Leer etiquetas YOLO
            with open(output_label_path_flip, "r") as f:
                lines = f.readlines()

            boxes = []
            class_labels = []
            for line in lines:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center, y_center, w, h = map(float, parts[1:])
                boxes.append([x_center, y_center, w, h])
                class_labels.append(class_id)


            # Aplicar espejo vertical (solo espejo)
            flipped_img = flip_augmenter_vertical(image=img_both)

            # Ajustar bounding boxes para el espejo
            flipped_boxes = [[x, 1.0 - y, w, h] for x, y, w, h in boxes]

            # Guardar imagen con solo el espejo
            output_img_name_flip = f"flipV_{output_img_name_flip}"
            output_img_path_flip = os.path.join(output_class_images, output_img_name_flip)
            cv2.imwrite(output_img_path_flip, cv2.cvtColor(flipped_img, cv2.COLOR_RGB2BGR))

            # Guardar etiquetas YOLO ajustadas para el espejo
            output_label_name_flip = output_img_name_flip.replace(".jpg", ".txt").replace(".png", ".txt")
            output_label_path_flip = os.path.join(output_class_labels, output_label_name_flip)
            with open(output_label_path_flip, "w") as f:
                for class_id, (x, y, w, h) in zip(class_labels, flipped_boxes):
                    f.write(f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

            
            # Aplicar espejo vertical (solo espejo)
            flipped_img = flip_augmenter_vertical(image=img)

            # Ajustar bounding boxes para el espejo
            flipped_boxes = [[x, 1.0 - y, w, h] for x, y, w, h in boxes]

            # Guardar imagen con solo el espejo
            output_img_name_flip = f"flipV_{img_name}"
            output_img_path_flip = os.path.join(output_class_images, output_img_name_flip)
            cv2.imwrite(output_img_path_flip, cv2.cvtColor(flipped_img, cv2.COLOR_RGB2BGR))

            # Guardar etiquetas YOLO ajustadas para el espejo
            output_label_name_flip = output_img_name_flip.replace(".jpg", ".txt").replace(".png", ".txt")
            output_label_path_flip = os.path.join(output_class_labels, output_label_name_flip)
            with open(output_label_path_flip, "w") as f:
                for class_id, (x, y, w, h) in zip(class_labels, flipped_boxes):
                    f.write(f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

print("Data Augmentation (Espejo Horizontal) completado y guardado en:", output_dir)


Procesando Lettuce en test: 100%|██████████| 38/38 [00:02<00:00, 16.89it/s]

Data Augmentation (Espejo Horizontal) completado y guardado en: augmented_data


In [43]:
import os
import cv2

# Función para dibujar el cuadro delimitador en la imagen
def draw_bounding_box(img, box):
    # Convertir las coordenadas normalizadas a píxeles
    height, width, _ = img.shape
    x_center, y_center, w, h = box
    x1 = int((x_center - w / 2) * width)  # esquina izquierda
    y1 = int((y_center - h / 2) * height)  # esquina superior
    x2 = int((x_center + w / 2) * width)  # esquina derecha
    y2 = int((y_center + h / 2) * height)  # esquina inferior
    
    # Dibujar el cuadro delimitador en la imagen
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    return img

# Ruta a la imagen y la etiqueta (ajustar estas rutas a tu caso)
img_path = 'C:\\Users\\Javie\\Desktop\\Master\\Desarrollo IA\\augmented_data\\test\\Lettuce\\Images\\flipV_flipH_38_JPG_jpg.rf.ea609e4b28b4cf826a5eb453e557e1d4.jpg'  # Cambia esta ruta a la imagen que quieres comprobar
label_path = 'C:\\Users\\Javie\\Desktop\\Master\\Desarrollo IA\\augmented_data\\test\\Lettuce\\Labels\\flipV_flipH_38_JPG_jpg.rf.ea609e4b28b4cf826a5eb453e557e1d4.txt'  # Cambia esta ruta al archivo de etiquetas correspondiente

# Cargar la imagen
img = cv2.imread(img_path)
if img is None or not os.path.exists(label_path):
    print("Error: imagen o archivo de etiqueta no encontrados.")
else:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Leer las etiquetas YOLO
    with open(label_path, "r") as f:
        lines = f.readlines()

    # Dibujar los cuadros delimitadores para cada etiqueta
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])  # No lo estamos utilizando, pero puedes mostrarlo si lo deseas
        x_center, y_center, w, h = map(float, parts[1:])
        box = [x_center, y_center, w, h]

        # Dibujar el cuadro delimitador sobre la imagen
        img_with_box = draw_bounding_box(img, box)

    # Mostrar la imagen con el cuadro delimitador
    cv2.imshow("Imagen con Bounding Box", cv2.cvtColor(img_with_box, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)  # Esperar a que el usuario presione una tecla para cerrar

cv2.destroyAllWindows()  # Cerrar la ventana de OpenCV


In [41]:
import os
import cv2

# Función para dibujar el cuadro delimitador en la imagen
def draw_bounding_box(img, box):
    # Convertir las coordenadas normalizadas a píxeles
    height, width, _ = img.shape
    x_center, y_center, w, h = box
    x1 = int((x_center - w / 2) * width)  # esquina izquierda
    y1 = int((y_center - h / 2) * height)  # esquina superior
    x2 = int((x_center + w / 2) * width)  # esquina derecha
    y2 = int((y_center + h / 2) * height)  # esquina inferior
    
    # Dibujar el cuadro delimitador en la imagen
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    return img

# Ruta a la imagen y la etiqueta (ajustar estas rutas a tu caso)
img_path = 'C:\\Users\\Javie\\Desktop\\Master\\Desarrollo IA\\test\\Lettuce\\Images\\38_JPG_jpg.rf.ea609e4b28b4cf826a5eb453e557e1d4.jpg'  # Cambia esta ruta a la imagen que quieres comprobar
label_path = 'C:\\Users\\Javie\\Desktop\\Master\\Desarrollo IA\\test\\Lettuce\\Labels\\38_JPG_jpg.rf.ea609e4b28b4cf826a5eb453e557e1d4.txt'  # Cambia esta ruta al archivo de etiquetas correspondiente

# Cargar la imagen
img = cv2.imread(img_path)
if img is None or not os.path.exists(label_path):
    print("Error: imagen o archivo de etiqueta no encontrados.")
else:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Leer las etiquetas YOLO
    with open(label_path, "r") as f:
        lines = f.readlines()

    # Dibujar los cuadros delimitadores para cada etiqueta
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])  # No lo estamos utilizando, pero puedes mostrarlo si lo deseas
        x_center, y_center, w, h = map(float, parts[1:])
        box = [x_center, y_center, w, h]

        # Dibujar el cuadro delimitador sobre la imagen
        img_with_box = draw_bounding_box(img, box)

    # Mostrar la imagen con el cuadro delimitador
    cv2.imshow("Imagen con Bounding Box", cv2.cvtColor(img_with_box, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)  # Esperar a que el usuario presione una tecla para cerrar

cv2.destroyAllWindows()  # Cerrar la ventana de OpenCV
